# Este cuaderno revisa los canales saturados y los de respuesta
 
## ADVERTENCIA: este cuaderno es potencialmente peligroso: 
## SOBRESCRIBE los datos "CanalesSaturados" y "Canalesrespuesta"


Vamos a reprocesar los hdf5 si es necesario con estas herramientas.

In [1]:
push!(LOAD_PATH, ".")
using PreprocTools
using HDF5, PyPlot, Statistics

┌ Info: Recompiling stale cache file /home/karel/.julia/compiled/v1.0/PreprocTools.ji for PreprocTools [top-level]
└ @ Base loading.jl:1190


In [20]:
arxname="/home/karel/RGutWork/JLDFiles/Desinhibida/180116s1/test_1.h5"
arx=h5open(arxname, "r+") # r+ :=read and append o algo asi"
names(arx)

10-element Array{String,1}:
 "CanalesBurbuja"  
 "CanalesRuido"    
 "CanalesSaturados"
 "Canalesrespuesta"
 "KTT_KInv"        
 "LFPTotal"        
 "Nota"            
 "Saturados"       
 "freq"            
 "kCSDCorrecta"    

In [3]:
LFP=read(arx["LFPTotal"]);
freq=read(arx["freq"]);
Saturados=read(arx["CanalesSaturados"])
Respuesta=read(arx["Canalesrespuesta"])
nota=read(arx["Nota"])

"Usamos las siguientes B y BT para obtener las Ks: BceroDura-r-0.25.dat y BtildeceroDura-r-0.25.dat"

In [4]:
size(Saturados)

(13, 2)

In [5]:
#los parametros son datos promediados, frecuencia, tiempo post estimulo en ms, umbral en microvolts, umbral de saturación en microvolts
#PruebaRespuesta=BCRP(DatosCentrados,freq, 0.1,100,-100, -300)
PruebaRespuesta=BuscaCanalRespActPot(LFP,freq, 0.1,100,-85, -300, 11, 24 )
#
# y luego los saturados
iniciobusqueda=0.1
finbusqueda=100
Saturados2=BuscaSaturados(LFP,freq, 800,iniciobusqueda,finbusqueda)
OtrosSaturados=BuscaSaturadosStd(LFP, 0.1,100, freq, 26)
yOtrosMas=BuscaRuidosos(LFP, 0.1, 100, freq, 100, 3)
ns=size(Saturados2)[1]
ns2=size(OtrosSaturados)[1]
ns3=size(yOtrosMas)[1]

nr=size(PruebaRespuesta)[1]
println("")
println("Encontramos ", ns, " canales probablemente saturados.")
println("Encontramos ", ns2, " canales probablemente con burbuja saturados.")
println("Encontramos ", ns3, " canales probablemente con ruido positivo.")
println("Encontramos ",  nr, " canales probablemente con actividad.")


Estoy buscando del cuadro 1 al , 703

Encontramos 14 canales probablemente saturados.
Encontramos 90 canales probablemente con burbuja saturados.
Encontramos 8 canales probablemente con ruido positivo.
Encontramos 73 canales probablemente con actividad.


## Aquí comienza la parte peligrosa. A Partir de aquí sobreescribimos los datos.

Es posible usar una copia del archivo si no esta Su Merce segura de lo que está haciendo. 
Correr la siguientes celdas SOBREESCRIBIRA los datos etiquetados como "CanalesSaturados" y "CanalesRespuesta"

In [17]:
if exists(arx, "CanalesSaturados") 
    o_delete(arx, "CanalesSaturados")
end
write(arx, "CanalesSaturados", Saturados2)


In [16]:
if exists(arx, "Canalesrespuesta")
   o_delete(arx, "Canalesrespuesta")
end
write(arx, "Canalesrespuesta", PruebaRespuesta)


yeah

 Estos probablemente sean nuevos para la gran mayoría de los archivos. (Criterios mas estrictos)

In [18]:
if exists(arx, "CanalesBurbuja")
   o_delete(arx, "CanalesBurbuja")
end
write(arx, "CanalesBurbuja", OtrosSaturados)

if exists(arx, "CanalesRuido")
   o_delete(arx, "CanalesRuido")
end
write(arx, "CanalesRuido", yOtrosMas)



In [19]:
close(arx)